### iterable
Any object from which the iter built-in function can obtain an iterator. Objects
implementing an \__iter__ method returning an iterator are iterable. Sequences
are always iterable, as are objects implementing a \__getitem__ method that
accepts 0-based indexes.

In [1]:
#Some people, when confronted with a problem, think "I know, I'll use regular expressions." 
#Now they have two problems.
import re
import reprlib

RE_WORD = re.compile(r'\w+')
RE_WORD.findall('1 2 3')

['1', '2', '3']

In [2]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, index):
         return self.words[index]
        
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) #for abbreviation

In [3]:
s = Sentence('"the time has come," the Walrus said,')
s

Sentence('"the time ha... Walrus said,')

In [4]:
#whenever python needs to iterate over an object x, it automatically calls iter(x)
#if __iter__ is missing iter() will call __getitem__()
for word in s:
    print(word)

the
time
has
come
the
Walrus
said


In [5]:
list(s)

['the', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [6]:
set(s)

{'Walrus', 'come', 'has', 'said', 'the', 'time'}

In [7]:
from collections import abc

class GooseSpam:
    def __iter__(self):
        pass


goose_spam_can = GooseSpam()

issubclass(GooseSpam, abc.Iterable), isinstance(goose_spam_can, abc.Iterable)

(True, True)

In [8]:
import random

def d6():
    return random.randint(1, 6)

In [9]:
d6_iter = iter(d6, 1) 
next(d6_iter), type(d6_iter)

(2, callable_iterator)

In [10]:
for roll in d6_iter:
    print(roll)

3
3
2
5


In [11]:
s = 'ABC'
it = iter(s) 

while True:
    try:
        print(next(it)) 
    except StopIteration: 
        del it 
        break 

A
B
C


### Python’s standard interface for an iterator has two methods:
\__next__:
Returns the next item in the series, raising StopIteration if there are no more.

\__iter__:
Returns self; this allows iterators to be used where an iterable is expected, for
example, in a for loop.

In [12]:
s3 = Sentence('Life of Brian')
it = iter(s3)

In [13]:
next(it), next(it), next(it)

('Life', 'of', 'Brian')

In [14]:
list(it)

[]

In [15]:
list(iter(s3))

['Life', 'of', 'Brian']

In [16]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [17]:
s4 = Sentence('hello good bye hi firend')
it = iter(s4)
next(it), list(it)

('hello', ['good', 'bye', 'hi', 'firend'])

A common cause of errors in building iterables and iterators is to confuse the two. To
be clear: iterables have an \__iter__ method that instantiates a new iterator every
time. Iterators implement a \__next__ method that returns individual items, and an
\__iter__ method that returns self.

To “support multiple traversals,” it must be possible to obtain multiple independent
iterators from the same iterable instance, and each iterator must keep its own internal
state, so a proper implementation of the pattern requires each call to iter(my_itera
ble) to create a new, independent, iterator. That is why we need the SentenceItera
tor class in this example

In [18]:
#just ignore this one
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return iter(self.words)

In [19]:
s3 = Sentence('hallo good bye hi')
it = iter(s3)
next(it), list(it)

('hallo', ['good', 'bye', 'hi'])

In [20]:
class Sentence2:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        for word in self.words:
            yield word       

In [21]:
s = Sentence2('hallo good bye hi')
for w in s:
    print(w)
    
it = iter(s)
next(it), list(it)

hallo
good
bye
hi


('hallo', ['good', 'bye', 'hi'])

In [22]:
#generators implement the iterator interface, so they are also iterable

In [23]:
def gen_123():
    yield 1
    yield 2
    yield 3
    
gen_123, gen_123()

(<function __main__.gen_123()>,
 <generator object gen_123 at 0x000001CB89A9EB90>)

In [24]:
for i in gen_123():
    print(i)

1
2
3


In [25]:
g = gen_123()
next(g), next(g), next(g)

(1, 2, 3)

In [26]:
#when the generator function returns it raises StopIteration(apparently return is implicit in python???)
next(g)

StopIteration: 

In [28]:
#the most lazy version of Sentence 
class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

In [29]:
class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

In [30]:
s = Sentence('1 2 3 hallo good bye')
it = iter(s)
next(it), list(it)

('1', ['2', '3', 'hallo', 'good', 'bye'])

In [31]:
for i in RE_WORD.finditer('1 2 3'):
    print(i)

<re.Match object; span=(0, 1), match='1'>
<re.Match object; span=(2, 3), match='2'>
<re.Match object; span=(4, 5), match='3'>


In [32]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')

In [33]:
res1 = [3*x for x in gen_AB()]

start
continue
end


In [34]:
res1

['AAA', 'BBB']

In [35]:
res2 = (3*x for x in gen_AB())
res2

<generator object <genexpr> at 0x000001CB8A09BC60>

In [36]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end


In [37]:
#generator expression
(2 * x for x in [1,2,3])

<generator object <genexpr> at 0x000001CB8A098D40>

In [38]:
type(8.0)(6)

6.0

In [39]:
def aritprog_gen(begin, step, end=None):
    result = type(begin+step)(begin)
    index = 0
    while (end is None) or result < end:
        yield result
        index += 1
        result = begin + step * index

In [40]:
list(aritprog_gen(1, .2, 3))

[1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4000000000000004, 2.6, 2.8]

In [41]:
from decimal import Decimal
list(aritprog_gen(1, Decimal('.2'), 3))

[Decimal('1'),
 Decimal('1.2'),
 Decimal('1.4'),
 Decimal('1.6'),
 Decimal('1.8'),
 Decimal('2.0'),
 Decimal('2.2'),
 Decimal('2.4'),
 Decimal('2.6'),
 Decimal('2.8')]

In [42]:
from fractions import Fraction
list(aritprog_gen(1, Fraction(1,3), 3))

[Fraction(1, 1),
 Fraction(4, 3),
 Fraction(5, 3),
 Fraction(2, 1),
 Fraction(7, 3),
 Fraction(8, 3)]

In [43]:
import itertools

In [44]:
gen = itertools.count(1, .5) #never stops so don't use list here
next(gen), next(gen), next(gen)

(1, 1.5, 2.0)

In [45]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [46]:
def aritprog(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is None:
        return ap_gen
    return itertools.takewhile(lambda n: n < end, ap_gen)

In [47]:
list(aritprog(0,1,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [48]:
from itertools import compress, dropwhile, accumulate, starmap, filterfalse, islice, takewhile

In [49]:
def vowel(c):
    return c.lower() in 'aeiou'

In [50]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [51]:
list(filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [52]:
list(takewhile(vowel, 'Aardvark'))

['A', 'a']

In [53]:
list(compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [54]:
list(islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [55]:
list(islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [56]:
list(islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']